In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import os

os.chdir('/content/gdrive/My Drive/특허경진대회/')
os.getcwd()



In [ ]:
data = pd.read_csv('data.csv')



In [ ]:
abstract = data['abstract']

In [ ]:
import re, os, string
import pandas as pd

# Scikit-learn importings
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def clean_text(text):
    """Doc cleaning"""
    
    # Lowering text
    text = text.lower()
    
    # Removing punctuation
    text = "".join([c for c in text if c not in PUNCTUATION])
    
    # Removing whitespace and newlines
    text = re.sub('\s+',' ',text)
    
    return text

In [ ]:
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform([doc])
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)
    
    return list(keywords.keys())

In [ ]:
# Constants
PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""" 
TOP_K_KEYWORDS = 10 # top k number of keywords to retrieve in a ranked document

In [ ]:
abstract = abstract.apply(clean_text)

In [ ]:
corpora = abstract.to_list()

In [ ]:
# Initializing TF-IDF Vectorizer with stopwords
vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True)

# Creating vocab with our corpora
# Exlcluding first 10 docs for testing purpose
vectorizer.fit_transform(corpora[10::])

# Storing vocab
feature_names = vectorizer.get_feature_names()

In [ ]:
result = []
for doc in corpora[0:10]:
    df = {}
    df['full_text'] = doc
    df['top_keywords'] = get_keywords(vectorizer, feature_names, doc)
    result.append(df)
    
final = pd.DataFrame(result)
final

In [ ]:
final[['full_text','top_keywords']]

In [ ]:
## Abstract keyword  wordcloud ##
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from wordcloud import WordCloud

In [ ]:
from collections import Counter

In [ ]:
top_keyword = sum(final['top_keywords'],[])

In [ ]:
count = Counter(top_keyword)

In [ ]:
words = dict(count.most_common())

In [ ]:
### 단어 추천 ###

In [ ]:
import re

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


stop_words=set(stopwords.words('english'))
nltk.download('punkt') # one time execution
nltk.download('stopwords')# one time execution

In [ ]:
tokenized_data = []
for sentence in abstract:
    temp_X = nltk.word_tokenize(sentence) 
    temp_X = [word for word in temp_X if not word in stop_words] 
    tokenized_data.append(temp_X)

In [ ]:
model_ft = FastText(tokenized_data, size=100, workers=4, sg=1, iter=6, word_ngrams=5)


In [ ]:
model_ft_df = pd.DataFrame(model_ft.wv.most_similar("data"), columns=['단어', '유사도'])

In [ ]:
model_ft_df